Based on article:

https://www.ibm.com/developerworks/community/blogs/jfp/entry/Python_Meets_Julia_Micro_Performance?lang=en&utm_content=buffer9bacd&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer

In [1]:
def fib(n):
    if n<2:
        return n
    return fib(n-1)+fib(n-2)

In [3]:
%timeit fib(20)

100 loops, best of 3: 2.5 ms per loop


#### Use Cython

In [11]:
%load_ext Cython

In [13]:
%%cython
def fib_cython(n):
    if n<2:
        return n
    return fib_cython(n-1)+fib_cython(n-2)

In [14]:
%timeit fib_cython(20)

1000 loops, best of 3: 1.03 ms per loop


#### Do even better with "STATIC TYPING"

In [16]:
%%cython
cpdef long fib_cython_type(long n):
    if n<2:
        return n
    return fib_cython_type(n-1)+fib_cython_type(n-2)

In [17]:
%timeit fib_cython_type(20)

10000 loops, best of 3: 40.1 µs per loop


> 60x FASTER THAN ORIGINAL!!

> One can argue that static typing defeats the purpose of Python.  I kind of agree with that in general, and we will see later a way to avoid this without sacrificing performance.  But I don't think this is the issue here.  The Fibonacci function is meant to be called with integers.  What we lose with static typing is the arbitrary precision that Python provides.  In the case of Fibonacci, using the C type long limits the size of the input parameter because too large parameters would result in integer overflow. 

#### Quicksort using Numpy

In [24]:
def qsort_kernel(a, lo, hi):
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel(a, lo, j)
        lo = i
        j = hi
    return a

In [25]:
import random
def benchmark_qsort():
    lst = [ random.random() for i in range(1,5000) ]
    qsort_kernel(lst, 0, len(lst)-1)

In [30]:
%timeit benchmark_qsort()

100 loops, best of 3: 11.1 ms per loop


In [32]:
%%cython
import numpy as np
cimport numpy as np
cpdef np.ndarray[double, ndim=1] \
qsort_kernel_cython_numpy_type(np.ndarray[double, ndim=1] a, \
                               long lo, \
                               long hi):
    cdef:
        long i, j
        double pivot
    i = lo
    j = hi
    while i < hi:
        pivot = a[(lo+hi) // 2]
        while i <= j:
            while a[i] < pivot:
                i += 1
            while a[j] > pivot:
                j -= 1
            if i <= j:
                a[i], a[j] = a[j], a[i]
                i += 1
                j -= 1
        if lo < j:
            qsort_kernel_cython_numpy_type(a, lo, j)
        lo = i
        j = hi
    return a

cpdef benchmark_qsort_numpy_cython():
    lst = np.random.rand(5000)
    qsort_kernel_cython_numpy_type(lst, 0, len(lst)-1)

In [33]:
%timeit benchmark_qsort_numpy_cython

10000000 loops, best of 3: 25.6 ns per loop


> We are about 15 times faster than the original benchmark, but this is still not the best way to use Python.  The best way is to use the Numpy built-in sort() function.  Its default behavior is to use the quick sort algorithm.  Timing this code

In [34]:
def benchmark_sort_numpy():
    lst = np.random.rand(5000)
    np.sort(lst)

In [36]:
%timeit benchmark_sort_numpy

10000000 loops, best of 3: 25 ns per loop
